In [2]:
from skimage.segmentation import watershed, felzenszwalb
from skimage.filters import sobel
import time
from skimage import io as skio
import pandas as pd
from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.filters import rank
from scipy import ndimage as ndi
from skimage.morphology import disk
import sklearn.metrics
import os
import torch

print('a')
def load_dataset(dataset_dir):
    dataset_list = []
    # Note: It's very important to load the images in the correct numerical order!
    # print(
    for image_file in list(sorted(Path(dataset_dir).glob("*.png"), key=lambda filename: int(filename.name.rstrip(".png")))):
        print(image_file)
        dataset_list.append(cv2.imread(str(image_file), cv2.IMREAD_GRAYSCALE))
    return np.stack(dataset_list, axis=0)

labels_train = pd.read_csv("Y_train.csv", index_col=0).T
data_train = load_dataset("X_train")
data_test = load_dataset("X_test")

a
X_train/0.png
X_train/1.png
X_train/2.png
X_train/3.png
X_train/4.png
X_train/5.png
X_train/6.png
X_train/7.png
X_train/8.png
X_train/9.png
X_train/10.png
X_train/11.png
X_train/12.png
X_train/13.png
X_train/14.png
X_train/15.png
X_train/16.png
X_train/17.png
X_train/18.png
X_train/19.png
X_train/20.png
X_train/21.png
X_train/22.png
X_train/23.png
X_train/24.png
X_train/25.png
X_train/26.png
X_train/27.png
X_train/28.png
X_train/29.png
X_train/30.png
X_train/31.png
X_train/32.png
X_train/33.png
X_train/34.png
X_train/35.png
X_train/36.png
X_train/37.png
X_train/38.png
X_train/39.png
X_train/40.png
X_train/41.png
X_train/42.png
X_train/43.png
X_train/44.png
X_train/45.png
X_train/46.png
X_train/47.png
X_train/48.png
X_train/49.png
X_train/50.png
X_train/51.png
X_train/52.png
X_train/53.png
X_train/54.png
X_train/55.png
X_train/56.png
X_train/57.png
X_train/58.png
X_train/59.png
X_train/60.png
X_train/61.png
X_train/62.png
X_train/63.png
X_train/64.png
X_train/65.png
X_train/66.png
X_t

In [3]:
print(len(data_train))

2000


In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="SAM/model.pth")
DEVICE = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
sam.to(device=DEVICE)
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
print(DEVICE)

In [ ]:
data_val = data_train
labels_val = labels_train

labels_val_predicted_SAM = []
t = time.time()
for i in range(1862, len(data_val)):
    rgb_img = np.stack((data_val[i],)*3, axis=-1)
    masks = mask_generator.generate(rgb_img)
    print(f"Image : {i}/{len(data_val)} time since beginning {time.time()-t}")
    label_im = np.zeros(data_val[0].shape)
    for j, mask in enumerate(masks):
        label_im[mask['segmentation']] = j
    np.save(os.path.join("SAM_masks", str(i)+".npy"), label_im)
    labels_val_predicted_SAM.append(label_im.flatten())
    # while 1: pass

In [ ]:
label_im = np.zeros(data_val[0].shape)
for i, mask in enumerate(masks):
    label_im[mask['segmentation']] = i
# print(len(masks))
# rand_index_dataset(labels_val, labels_val_predicted_baseline)
plt.imshow(label_im)
plt.no_axis()

In [ ]:
labels_val_predicted_SAM = []
for i in range(2000):
    labels_val_predicted_SAM.append(np.load(os.path.join("SAM_masks", str(i)+".npy")))

labels_val_cut = labels_val.iloc[:200]
labels_val_predicted_SAM_cut = labels_val_predicted_SAM[:200]

In [ ]:
def rand_index_dataset(y_true_df: pd.DataFrame, y_pred_df: pd.DataFrame) -> float:
    """Compute the individual rand index for each sample in the dataset and then average it"""
    individual_rand_index = []
    for row_index in range(y_true_df.values.shape[0]):
        labels = y_true_df.values[row_index]
        preds = y_pred_df.values[row_index]
        individual_rand_index.append(sklearn.metrics.adjusted_rand_score(labels, preds))

    return np.mean(individual_rand_index)

labels_val_predicted_SAM_df = pd.DataFrame(np.stack(labels_val_predicted_SAM_cut, axis=0).reshape((len(labels_val_predicted_SAM_cut), -1)))

print(labels_val_predicted_SAM_df.values)
rand_index_dataset(labels_val_cut, labels_val_predicted_SAM_df)